## 데이터 로드

In [1]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [2]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [3]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [4]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [5]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [6]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [7]:
train_data_sample = train_data[:500]

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [8]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head(30)

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861
5,[락],139541
6,[락],351214
7,[락],650298
8,[락],531057
9,[락],205238


In [9]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'댄스', '스트레스해소', 'kpop', '여자아이돌', '걸그룹댄스'}


In [10]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(16674, 2)

In [11]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [12]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [13]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16674 entries, 0 to 16673
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                16674 non-null  object
 1   song_id             16674 non-null  int64 
 2   gnr                 16674 non-null  object
 3   issue_date          16674 non-null  int64 
 4   album_id            16674 non-null  int64 
 5   artist_id_basket    16674 non-null  object
 6   song_name           16674 non-null  object
 7   artist_name_basket  16674 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.1+ MB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [14]:
train_data_sample2 = train_data[:500]

In [15]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 1, workers = 4, sg = 1)

w2v.wv.vectors.shape

(727, 100)

In [16]:
print(w2v.wv.most_similar('락'))

[('새벽', 0.38008394837379456), ('방학', 0.323152095079422), ('기분전환', 0.3221883475780487), ('캐롤', 0.26910120248794556), ('대만', 0.2640238404273987), ('댄스', 0.263297438621521), ('청량한', 0.2608855664730072), ('어쿠스틱', 0.253212571144104), ('우울', 0.24505077302455902), ('명곡', 0.24368607997894287)]


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(16674, 194)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.70710678,
        0.70710678],
       [0.        , 0.        , 0.        , ..., 0.70710678, 1.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 0.5       ,
        1.        ]])

In [19]:
simi_test = gnr_sim[1, :]
simi_test2 = gnr_sim[2, :]
print(simi_test + simi_test2)
print(song_data[song_data['song_id']==525514])
print(song_data[song_data['song_id']==129701])
print(song_data[song_data['song_id']==229622])

[0.         1.33333333 1.33333333 ... 0.         0.         0.        ]
                     gnr  issue_date  album_id artist_id_basket  \
525514  [GN1402, GN1401]    20130506   2200223         [734201]   

              song_name artist_name_basket  song_id  
525514  Hey Little Girl          [The Sol]   525514  
                             gnr  issue_date  album_id artist_id_basket  \
129701  [GN0901, GN0902, GN1001]    20130917   2201802         [536907]   

       song_name artist_name_basket  song_id  
129701   Octagon      [Royal Bangs]   129701  
                                     gnr  issue_date  album_id  \
229622  [GN1402, GN1401, GN0901, GN0902]    20061212    528351   

       artist_id_basket  song_name artist_name_basket  song_id  
229622         [406781]  Say Hello     [Rosie Thomas]   229622  


- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [20]:
import numpy as np

def find_sim_song(df, sim_matrix, songs, top_n=10):
    simi = np.zeros(len(df['song_id']))
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        simi = simi + sim_matrix[title_index, :]
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    
    #for song in songs:
    #    title_song = df[df['song_id'] == song]
    #    title_index = title_song.index.values
    #    
    #    temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
    
    # 유사도가 0.5 이하인 경우는 제외
    #temp = temp[temp['similarity'] >= 0.5]
    
    temp = temp.reset_index(drop=True)

    #final_index = temp.index.values[ : top_n]
    
    return temp.iloc[ : top_n]

In [21]:
similar_songs = find_sim_song(song_tag_appended, gnr_sim, [525514, 129701, 229622], 10)

similar_songs[['song_id', 'similarity', 'issue_date', 'gnr']]

,song_id,similarity,issue_date,gnr
0,191430,0.761486,20140407,"[GN1402, GN1401, GN0901, GN0902]"
1,463782,0.761486,20170227,"[GN1402, GN1401, GN0901, GN0902]"
2,229622,0.761486,20061212,"[GN1402, GN1401, GN0901, GN0902]"
3,205238,0.606558,20110124,"[GN0904, GN1402, GN1401, GN0901, GN1001]"
4,701978,0.575727,20101011,"[GN1402, GN1401, GN1001]"
5,513731,0.569036,20111018,"[GN1402, GN1401]"
6,489449,0.569036,20111111,"[GN1402, GN1401]"
7,392798,0.569036,20111018,"[GN1402, GN1401]"
8,525514,0.569036,20130506,"[GN1402, GN1401]"
9,598147,0.569036,20131105,"[GN1402, GN1401]"


## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [22]:
'''
def song_recommend(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
        print(simi_songs)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 태그와 유사도 두 부분에서 동일한 개수(홀수일 경우 태그 > 유사도) 추출
        # sc = 유사도 부분에서 추출할 노래의 개수
        if len(recommended) % 2 == 0:
            sc = (10-len(recommended)) / 2
        else:
            sc = (10-len(recommended)) // 2
        
        # 태그는 있고 히스토리가 없는 경우 태그 부분에서 10개를 추출하기 위한 코드
        if len(songs) == 0:
            sc = 0
        
        # 이미 추출한 노래를 제외하고 태그 부분에서 정해진 개수만큼 추출한다
        # 태그가 없을 경우 동작하지 않음
        index = 0
        while len(tag_songs) != 0 and len(recommended) < (10 - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommended:
                recommended.append(tag_song)
            
            index += 1
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]
'''

"\ndef song_recommend(tags, songs, tag_df, song_df, sim_mat):\n    ts = tags\n    \n    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다\n    all_tags = []\n    if len(ts) != 0 and len(ts) < 3:\n        for tag in ts:\n            sim_tags = w2v.wv.most_similar(tag)\n            for t in sim_tags:\n                all_tags.append(t)\n        all_tags = sorted(all_tags, key = lambda x : -x[1])\n        i = 0\n        while len(ts) != 0 and len(ts) < 3:\n            tag = all_tags[i][0]\n            if tag not in ts:\n                ts.append(tag)\n                i += 1\n\n    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다\n    tag_songs = dict()\n    \n    for tag in ts:\n        for i in range(len(tag_df['song_id'])):\n            if tag in tag_df['tags'][i]:\n                \n                for ss in tag_df['song_id'][i]:\n                    if not ss in songs:\n                        \n                        if ss in tag_songs:\n                            tag_songs[ss] += 1\n  

In [23]:
'''
def song_recommend_test(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
    
    
    
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10 and len(simi_songs) != 0 and len(simi_songs) > index:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
        if len(recommended) < 10:
            index = 0
            while len(tag_songs) != 0 and len(recommended) < 10:
                tag_song = tag_songs[index][0]
                if not tag_song in recommended:
                    recommended.append(tag_song)
                index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]
'''

"\ndef song_recommend_test(tags, songs, tag_df, song_df, sim_mat):\n    ts = tags\n    \n    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다\n    all_tags = []\n    if len(ts) != 0 and len(ts) < 3:\n        for tag in ts:\n            sim_tags = w2v.wv.most_similar(tag)\n            for t in sim_tags:\n                all_tags.append(t)\n        all_tags = sorted(all_tags, key = lambda x : -x[1])\n        i = 0\n        while len(ts) != 0 and len(ts) < 3:\n            tag = all_tags[i][0]\n            if tag not in ts:\n                ts.append(tag)\n                i += 1\n\n    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다\n    tag_songs = dict()\n    \n    for tag in ts:\n        for i in range(len(tag_df['song_id'])):\n            if tag in tag_df['tags'][i]:\n                \n                for ss in tag_df['song_id'][i]:\n                    if not ss in songs:\n                        \n                        if ss in tag_songs:\n                            tag_songs[ss] += 

# 모델 평가
- [참고](https://chrisjune-13837.medium.com/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80%EB%B0%A9%EB%B2%95-with-python-9932097f0ff9)

# Recall@K

In [24]:
def get_recall_k(y_true, y_pred):
    recall_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(true_items) if len(true_items) > 0 else 0
    return recall

#recall_k = get_recall_k(my_songs1, pred_list)
#print("Recall@K (K=10): {:.2f}".format(recall_k))

# Precision@K

In [25]:
def get_precision_k(y_true, y_pred):
    precision_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(pred_items) if len(pred_items) > 0 else 0
    return recall

#precision_k = get_precision_k(my_songs1, pred_list)
#print("Precision@K (K=10): {:.2f}".format(precision_k))

# Average Precision@K
- 아이템 개수가 매우 적은 경우, 모델의 성능 또한 낮아지는 문제가 발생
    - 해결방법
        1. Thresholding : 사용자 플레이리스트 내 음악이 일정 개수 이상인 경우만 적용
        2. Item Weighting : hits 에 1이 아닌 가중치를 이용. 이 때 precision 이 1 보다 크지 않게 하기 위해, 정규화 필요

In [26]:
def get_ap_k(y_true, y_pred, k):
    pred_items = y_pred[:k]
    hits = []
    for item in pred_items:
        if item in y_true:
            hits.append(1)
        else:
            hits.append(0)
    precision_values = []
    for i in range(1, k+1):
        precision_values.append(sum(hits[:i]) / i)
        
    print(precision_values)
    
    if len(precision_values) == 0:
        return 0
    else:
        return sum(precision_values) / len(precision_values)
    
#ap_k = get_ap_k(my_songs1, pred_list, 10)
#print("AP@K (K=10): {:.2f}".format(ap_k))

# Mean Average Precision@K

In [27]:
def get_map_k(y_true, y_pred, k):
    sum_ap = 0
    for true_item, pred_item in zip(y_true, y_pred):
        ap_k = get_ap_k(true_item, pred_item, k)
        sum_ap += ap_k
    if len(y_true) == 0:
        return 0
    else:
        return sum_ap / len(y_true)
    
#map_k = get_map_k(all_my_songs, all_pred_songs, 10)
#print("MAP@K (K=10): {:.2f}".format(map_k))

In [28]:
train_data.head(10)

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."
5,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[418694, 222305, 96545, 135950, 304687, 457451..."
6,"[짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]","[528130, 505036, 398947, 627363, 37748, 376358..."
7,"[잔잔한, 추억, 회상]","[496913, 632529, 501426, 515574, 411161, 10341..."
8,"[일렉트로니카, 포크, 메탈, 락, 댄스, 인디]","[634861, 270738, 163936, 692209, 449477, 56342..."
9,"[록, Metal, 이일우, M에센셜, 메탈, Rock, 락]","[613315, 439294, 230806, 497932, 113479, 93506..."


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_cv(songs):
    songs['gnr_literal'] = songs['gnr'].apply(lambda x : (' ').join(x))
    count_vect = CountVectorizer()
    gnr_mat = count_vect.fit_transform(songs['gnr_literal'])
    return gnr_mat

def get_sim(gnr_mat):
    gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
    return gnr_sim

# 추천 함수 수정

### 사용자 플레이리스트 내 음악의 태그를 포함하는 노래 중에서 장르 유사도를 측정

In [30]:
def song_recommend(tags, songs, tag_df, song_df):
    
    ts = tags
    '''
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1
    '''
                
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        #print("현재 태그: {}".format(tag))
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                #print("태그에 맞는 노래: {}".format(tag))
                for ss in tag_df['song_id'][i]:
                    #print(ss)
                    if ss in tag_songs:
                        tag_songs[ss] += 1
                            
                    else:
                        tag_songs[ss] = 1
                        
    #tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    tag_songs = {key: value for key, value in tag_songs.items() if value > 0}
    tag_songs = list(tag_songs)
    tag_songs = song_df[song_df['song_id'].isin(tag_songs)]
    tag_songs = tag_songs.reset_index(drop=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        sim_mat = get_sim(get_cv(tag_songs))
        simi_songs = find_sim_song(tag_songs, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    
    for rec in simi_songs['song_id']:
        title_song = tag_songs[tag_songs['song_id'] == rec]
        title_index = title_song.index
        recommended.append(title_index[0])
    
    return tag_songs.iloc[recommended[:10]]



### 사용자 플레이리스트 내 음악과 높은 유사도를 가진 곡들 중, 태그 유사도가 높은 곡을 추천
- 높은 유사도를 가진 곡을 상위 100개로 할지, 일정 유사도보다 높은 경우로 할지를 나눠서 테스트할 예정

In [31]:
def get_tag_simi(input_tag, simi_tags):
    simi_lst = [w2v.wv.similarity(input_tag, simi_tag) for simi_tag in simi_tags]
    return max(simi_lst)

def song_recommend2(tags, songs, tag_df, song_df):
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        sim_mat = get_sim(get_cv(song_df))
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    #print(simi_songs)
    
    '''

    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1
    '''
       
    ts = tags
    
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    tag_simi_mean = []
    
    for simi_tags in simi_songs['tags']:
        songs_similality = []
        for input_tag in ts:
            songs_similality.append(get_tag_simi(input_tag, simi_tags))
        tag_simi_mean.append(sum(songs_similality) / len(songs_similality))
    
    tag_simi_mean = pd.DataFrame(columns = ['tag_simi'], data = tag_simi_mean)
    sorted_idx = tag_simi_mean['tag_simi'].sort_values(ascending = False).index
    #for i in sorted_idx:
    #    print("원본: {} / simi_mean: {}".format(simi_songs.loc[i, 'song_id'], tag_simi_mean.loc[i, 'tag_simi']))
    tag_simi_mean = simi_songs.loc[sorted_idx]
    
    '''
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    
    for rec in simi_songs['song_id']:
        title_song = tag_songs[tag_songs['song_id'] == rec]
        title_index = title_song.index
        recommended.append(title_index[0])
    
    '''
    
    return tag_simi_mean.iloc[:10]



In [32]:
test_data_sample = train_data[:500]
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()

rec1 = song_recommend(test_my_tags[0], test_my_songs[0], train_data_sample2, song_tag_appended)

pred_list = rec1['song_id'].tolist()
precision_k = get_precision_k(test_my_songs[0], pred_list)
recall_k = get_recall_k(test_my_songs[0], pred_list)

print("Recall@K (K=10): {:.2f}".format(recall_k))
print("Precision@K (K=10): {:.2f}".format(precision_k))

Recall@K (K=10): 0.16
Precision@K (K=10): 0.30


In [33]:
rec2 = song_recommend2(test_my_tags[0], test_my_songs[0], train_data_sample2, song_tag_appended)
rec2.head(10)

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,gnr_literal,similarity
76,[락],178323,"[GN0901, GN0902, GN1001]",20110125,1187354,[223484],Tree By The River,[iron & wine],GN0901 GN0902 GN1001,0.477692
77,[락],592196,"[GN0901, GN0902, GN1001]",20110222,2137908,[101138],You Only Live Once,[The Strokes],GN0901 GN0902 GN1001,0.477692
44,"[빗물같은, 락, 비오는날]",541824,"[GN0901, GN0902, GN1001]",20121112,2166466,[597831],Yayo,[Lana Del Rey],GN0901 GN0902 GN1001,0.477692
32,"[록, 신나는, 스타일, 락, Rock, 밴드, 기분업, 세련된]",423626,"[GN0901, GN0902, GN1001]",20130422,2182449,[19035],Entertainment,[Phoenix],GN0901 GN0902 GN1001,0.477692
33,"[록, 신나는, 스타일, 락, Rock, 밴드, 기분업, 세련된]",75991,"[GN0901, GN0902, GN1001]",20130930,2203239,[459133],Changing Of The Seasons,[Two Door Cinema Club],GN0901 GN0902 GN1001,0.477692
35,"[힐링, 팝, 인디, 가요, 사랑, 일상, 새벽, 감성, 비오는날, 주말, 밤, 겨울]",599123,"[GN0901, GN0902, GN1001]",20130506,2184051,[247867],Step,[Vampire Weekend],GN0901 GN0902 GN1001,0.477692
82,"[새벽, 밤, 잔잔한]",85335,"[GN0901, GN0902, GN1001]",20130129,2173759,[58796],Cruisin` (Solo),[Sioen],GN0901 GN0902 GN1001,0.477692
64,"[새벽, 밤, 잔잔한]",375010,"[GN0901, GN0902, GN1001]",20161007,2684525,[231983],Colors,[OneRepublic],GN0901 GN0902 GN1001,0.477692
85,"[새벽, 밤, 비오는날]",467570,"[GN0901, GN0902, GN1001]",20130313,2177904,[474562],Help,[Hurts],GN0901 GN0902 GN1001,0.477692
58,"[팝, 기분전환, 빌보드, 명곡, 믿고듣는, 드라이브]",252277,"[GN0901, GN0902, GN1001]",20120625,2130333,[107721],Moves Like Jagger (Studio Recording From The V...,[Maroon 5],GN0901 GN0902 GN1001,0.477692


In [34]:
print(rec1)

                                                  tags  song_id  \
186                                    [빗물같은, 락, 비오는날]   541824   
88   [느껴져, 파티, 봄, 기분전환, 여행, 스트레스, 신나는, 드라이브, 팝, 댄스,...   562575   
13                                                 [락]   461973   
1                                                  [락]   129701   
209               [록, 신나는, 스타일, 락, Rock, 밴드, 기분업, 세련된]    75991   
229                                                [락]   178323   
237                                                [락]   592196   
207               [록, 신나는, 스타일, 락, Rock, 밴드, 기분업, 세련된]   423626   
5                                                  [락]   139541   
327     [실험, 인디, 1990, 영국, 드림팝, 락, 기타, 슈게이징, 센치한날, 몽환]   266993   

                                  gnr  issue_date  album_id artist_id_basket  \
186          [GN0901, GN0902, GN1001]    20121112   2166466         [597831]   
88           [GN0901, GN0902, GN1001]    20120625   2130333         [107721]   
13           [GN0901, 

In [35]:
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend(tags, plist, train_data_sample2, song_tag_appended)
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])

[541824, 562575, 461973, 129701, 75991, 178323, 592196, 423626, 139541, 266993]


In [36]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[0.0, 0.0, 0.3333333333333333, 0.5, 0.4, 0.3333333333333333, 0.2857142857142857, 0.25, 0.3333333333333333, 0.3]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.3333333333333333, 0.25, 0.4, 0.3333333333333333, 0.2857142857142857, 0.375, 0.4444444444444444, 0.5]
[0.0, 0.0, 0.0, 0.25, 0.4, 0.3333333333333333, 0.42857142857142855, 0.375, 0.3333333333333333, 0.3]
[0.0, 0.5, 0.6666666666666666, 0.75, 0.8, 0.6666666666666666, 0.7142857142857143, 0.75, 0.7777777777777778, 0.8]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.125, 0.1111111111111111, 0.1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.5, 0.3333333333333333, 0.25, 0.4, 0.5, 0.5714285714285714, 0.625, 0.5555555555555556, 0.6]
[1.0, 0.5, 0.6666666666666666, 0.5, 0.4, 0.5, 0.42857142857142855, 0.375, 0.3333333333333333, 0.4]
[0.0, 0.5, 0.6666666666666666, 0.75, 0.8, 0.8333333333333334, 0.8571428571428571, 0.875, 0.8888888888888888, 0.9]
[0.0, 0.0, 0

In [37]:
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend2(tags, plist, train_data_sample2, song_tag_appended)
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])

[178323, 592196, 541824, 423626, 75991, 599123, 85335, 375010, 467570, 252277]


In [38]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.75, 0.6, 0.5, 0.42857142857142855, 0.375, 0.3333333333333333, 0.3]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.5, 0.3333333333333333, 0.5, 0.6, 0.6666666666666666, 0.7142857142857143, 0.75, 0.7777777777777778, 0.8]
[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 0.7142857142857143, 0.625, 0.5555555555555556, 0.5]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571, 0.75, 0.6666666666666666, 0.6]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.5, 0.3333

# nDCG

In [56]:
def ndcg_at_k(y_true, y_pred, k):
    ndcg = 0
    ranking = []
    
    for i in range(k):
        if y_pred[i] in y_true:
            ranking.append(1)
        else:
            ranking.append(0.1)
    print(ranking)
        
    # Ideal ranking을 계산하기 위해 ranking을 내림차순으로 정렬한 별도의 리스트
    ideal_ranking = sorted(ranking, reverse=True)
    
    # DCG 계산
    dcg = ranking[0]
    for i in range(1, min(k, len(ranking))):
        dcg += ranking[i] / np.log2(i + 1)
    
    # Ideal DCG 계산
    ideal_dcg = ideal_ranking[0]
    for i in range(1, min(k, len(ideal_ranking))):
        ideal_dcg += ideal_ranking[i] / np.log2(i + 1)
    
    # nDCG 계산
    if ideal_dcg == 0:
        ndcg = 0
    else:
        ndcg = dcg / ideal_dcg
    
    return ndcg

In [57]:
ndcg = 0
for true_item, pred_item in zip(test_my_songs, pred_list):
    ndcg += ndcg_at_k(true_item, pred_item, k=10)
    
ndcg_result = ndcg / len(test_my_songs)
print(f'nDCG: {ndcg_result:.4f}')

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 1, 0.1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 0.1, 1, 0.1, 1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,